In [1]:
import os
import utilities
from maths_objects import (
    Formulas, SiteHierarchies,MarkdownFile, DataSource, FrontMatter, 
    FormulaTable)
import pandas as pd


data_sources = DataSource()
site_hierarchy = SiteHierarchies(data_sources.site_hierarchies())


In [2]:
site_hierarchy.all_path_levels()

['NSW/Formulas/By year/Year 9',
 'NSW/Formulas/By year',
 'NSW/Formulas/By year cumulative/Year 12 Adv',
 'NSW/Topics/By year/Year 12 Adv',
 'NSW/Topics/By year cumulative/Year 9',
 'NSW/Formulas/By year/Year 12 Adv/Summaries',
 'NSW/Topics/By year/Year 9',
 'NSW/Formulas/By year/Year 12 Ext 1/Summaries',
 'NSW/Formulas/By year cumulative/Year 12 Adv/Summaries',
 'NSW/Formulas/By year/Year 12 Ext 1',
 'NSW/Topics/By year/Year 11 Adv',
 'NSW/Topics/By year cumulative/Year 11 Adv',
 'NSW/Formulas',
 'NSW/Formulas/By year cumulative',
 'NSW/Topics/By year cumulative',
 'NSW/Topics',
 'NSW/Topics/By year',
 'NSW/Formulas/By year/Year 11 Adv',
 'NSW/Formulas/By year cumulative/Year 12 Ext 1',
 'NSW/Formulas/By year cumulative/Year 12 Ext 1/Summaries',
 'NSW',
 'NSW/Topics/By year cumulative/Year 12 Adv',
 'NSW/Formulas/By year/Year 12 Adv']

In [32]:
dirs = site_hierarchy.directory_paths()
dirs = [x.split(os.path.sep) for x in dirs]
df = pd.DataFrame(dirs)

display(df)


def all_path_levels(_hierarchy_data):
    """Returns a list of unique paths recursively at each directory level
    for the directory hierarchies stored in this class
    """
    all_path_levels = []
    for column_index in range(len(_hierarchy_data.columns)):
        paths_at_current_level = _hierarchy_data.iloc[
            :, :column_index+1]
        paths_at_current_level = (paths_at_current_level
                                    .drop_duplicates().dropna())
        paths_at_current_level = paths_at_current_level.apply(
                os.path.sep.join, axis=1)
        all_path_levels = all_path_levels + list(paths_at_current_level)
    all_path_levels = list(set(all_path_levels))
    return all_path_levels


,0,1,2,3,4
0,NSW,Topics,By year,Year 9,None
1,NSW,Formulas,By year cumulative,Year 12 Adv,None
2,NSW,Formulas,By year,Year 11 Adv,None
3,NSW,Topics,By year,Year 11 Adv,None
4,NSW,Formulas,By year cumulative,Year 12 Ext 1,None
5,NSW,Topics,By year cumulative,Year 12 Adv,None
6,NSW,Topics,By year cumulative,Year 9,None
7,NSW,Formulas,By year,Year 12 Ext 1,None
8,NSW,Formulas,By year,Year 12 Adv,None
9,NSW,Topics,By year,Year 12 Adv,None


In [2]:
def create_formula_pages(formulas, base_dir):
    for formula_table in formulas.formula_tables():
        target_path = os.path.sep.join([
            base_dir, formula_table.state, 'Formulas', 'By year',
            formula_table.subject, 
            formula_table.category
        ])
        if not os.path.isdir(target_path):
            raise ValueError (target_path.replace(base_dir, '')
                              +' does not exist in site_hierarchy.csv')
        with open(target_path, "w", encoding="utf-8") as text_file:
            text_file.write(formula_table.to_markdown())

docs_dir = data_sources.docs_directory()
formulas_by_year = Formulas(data_sources.formulas_by_year())
create_formula_pages(formulas_by_year, docs_dir)

IsADirectoryError: [Errno 21] Is a directory: '/home/charl/Onedrive/Documents_Charl/Computer_Technical/Programming_GitHub/AustralianSchoolMaths/content/docs/NSW/Formulas/By year/Year 9/Indices'

In [6]:
formulas.filter_by_state_subject_category('NSW', 'Year 10', 'Indices').to_dataframe()

,State,Subject,Syllabus topic,Syllabus subtopic code,Syllabus subtopic,Category,Subcategory_1,Subcategory_2,Description,Group,Formula,On formula sheet,Proof required,Comment


In [4]:
formulas.filter_by_state_subject_category('NSW', 'Year 9', 'Indices')()

AttributeError: 'DataFrame' object has no attribute 'filter_by_function'

In [13]:
formulas.filter_by_function(
    lambda row : 
    (row['State'] == 'NSW') and 
    (row['Subject'] == 'Year 9') and 
    (row['Category'] == 'Indices')
    ).to_dataframe()

,State,Subject,Syllabus topic,Syllabus subtopic code,Syllabus subtopic,Category,Subcategory_1,Subcategory_2,Description,Group,Formula,On formula sheet,Proof required,Comment
0,NSW,Year 9,TBA,TBA,TBA,Indices,Index Laws,NaN,Multiplying terms with same base,NaN,$a^m \times a^n = a^{m+n}$,True,False,NaN
1,NSW,Year 9,TBA,TBA,TBA,Indices,Index Laws,NaN,Dividing terms with same base,NaN,$a^m \div a^n = \dfrac{a^m}{a^n} = a^{m-n}$,True,False,NaN
2,NSW,Year 9,TBA,TBA,TBA,Indices,Index Laws,NaN,Power of a power,NaN,$(a^m)^n = a^{m \times n}$,True,False,NaN
3,NSW,Year 9,TBA,TBA,TBA,Indices,Index Laws,NaN,Powers of products,NaN,$(ab)^n = a^nb^n$,True,False,NaN
4,NSW,Year 9,TBA,TBA,TBA,Indices,Index Laws,NaN,Powers of quotients,NaN,$\left(\dfrac{a}{b}\right)^n = \dfrac{a^n}{b^n}$,True,False,NaN
5,NSW,Year 9,TBA,TBA,TBA,Indices,Index Laws,NaN,Power of zero,NaN,$a^0=1$,True,False,NaN
6,NSW,Year 9,TBA,TBA,TBA,Indices,Index Laws,NaN,Negative powers,NaN,$a^{-n}=\dfrac{1}{a^n}$,True,False,NaN
7,NSW,Year 9,TBA,TBA,TBA,Indices,Index Laws,NaN,Negative powers of quotients,NaN,$\left(\dfrac{a}{b}\right)^{-n} = \left(\dfrac...,True,False,NaN
8,NSW,Year 9,TBA,TBA,TBA,Indices,Index Laws,NaN,Fractional powers,NaN,$a^{\frac{m}{n}} =\sqrt[n]{a^m}$,True,False,NaN


In [22]:

data_sources = DataSource()

formulas_by_year = data_sources.formulas_by_year()
state_subject_sort_orders = data_sources._state_subject_sort_orders()


# Add the subject Sort order (representing the sort order for the
# subject by given state) to the formulas data
return_data = state_subject_sort_orders.merge(
    right=formulas_by_year,
    how='inner',
    left_on=['Sort state', 'Sort subject'],
    right_on=['State', 'Subject']
)
return_data = return_data.rename(
    columns={
        'State subject sort order': 'Data sort index'})
return_data = return_data.drop(columns=['Sort state',
                                        'Sort subject'])

return_data = return_data.merge(
    right=state_subject_sort_orders, 
    how='inner', left_on=['State'], right_on=['Sort state']
)
return_data = return_data[return_data['State subject sort order'] >=
                        return_data['Data sort index']]
return_data = return_data.drop(columns=['Data sort index',
                                    'State subject sort order',
                                    'Sort state', 'Subject'])
return_data = return_data.rename(columns={
    'Sort subject': 'Subject'})


In [23]:
temp_file = data_sources.website_creator_directory() + os.path.sep + 'Temp.csv'

return_data.to_csv(temp_file)


In [2]:
data_sources = DataSource()
data_sources._state_subject_sort_orders()

,State,Subject,State subject sort order
0,NSW,Year 9,0
1,NSW,Year 11 Adv,1
2,NSW,Year 12 Adv,2
3,NSW,Year 12 Ext 1,3


In [2]:
a = my_dirs.truncate_to_path_length(4).unique().reset_sort_index()

display(a.to_dataframe())

b = a.filter_by_path_start(['NSW', 'Topics'])
display(b.to_dataframe())
display(a.to_dataframe())

,Level_0,Level_1,Level_2,Level_3
0,NSW,Topics,By year,Year 9
1,NSW,Topics,By year,Year 11 Adv
2,NSW,Topics,By year,Year 12 Adv
3,NSW,Topics,By year cumulative,Year 9
4,NSW,Topics,By year cumulative,Year 11 Adv
5,NSW,Topics,By year cumulative,Year 12 Adv
6,NSW,Formulas,By year,Year 9
7,NSW,Formulas,By year,Year 12 Adv
8,NSW,Formulas,By year,Year 12 Ext 1
9,NSW,Formulas,By year cumulative,Year 12 Adv


,Level_0,Level_1,Level_2,Level_3
0,NSW,Topics,By year,Year 9
1,NSW,Topics,By year,Year 11 Adv
2,NSW,Topics,By year,Year 12 Adv
3,NSW,Topics,By year cumulative,Year 9
4,NSW,Topics,By year cumulative,Year 11 Adv
5,NSW,Topics,By year cumulative,Year 12 Adv


,Level_0,Level_1,Level_2,Level_3
0,NSW,Topics,By year,Year 9
1,NSW,Topics,By year,Year 11 Adv
2,NSW,Topics,By year,Year 12 Adv
3,NSW,Topics,By year cumulative,Year 9
4,NSW,Topics,By year cumulative,Year 11 Adv
5,NSW,Topics,By year cumulative,Year 12 Adv
6,NSW,Formulas,By year,Year 9
7,NSW,Formulas,By year,Year 12 Adv
8,NSW,Formulas,By year,Year 12 Ext 1
9,NSW,Formulas,By year cumulative,Year 12 Adv


In [34]:

# Incorporate below in index file creation to make it clearer how levels are 
# iterated.  Can I improve further by creating an iterator.  In the 
# Web page hierarchy class?

dir_levels = school_formulas.by_year_dirs()
current_dir_levels = dir_levels
for column in reversed(dir_levels.columns):
    display(current_dir_levels)
    current_dir_levels = current_dir_levels.drop(columns=column)

,State,Formula subcategory 1,Formula subcategory 2,Subject
0,NSW,Formulas,By Year,Year 9
9,NSW,Formulas,By Year,Year 11 Adv
21,NSW,Formulas,By Year,Year 12 Adv
42,NSW,Formulas,By Year,Year 12 Ext 1


,State,Formula subcategory 1,Formula subcategory 2
0,NSW,Formulas,By Year
9,NSW,Formulas,By Year
21,NSW,Formulas,By Year
42,NSW,Formulas,By Year


,State,Formula subcategory 1
0,NSW,Formulas
9,NSW,Formulas
21,NSW,Formulas
42,NSW,Formulas


,State
0,NSW
9,NSW
21,NSW
42,NSW


In [7]:
def create_index_files(base_dir, dir_levels,site_hierarchies,  
                       front_matter=None):
    """Creates _index.md files recursively at each level of the directory
    structures formed by joining base_dir string and each element of 
    dir_levels.  The content of each file is set based  on the content
    of the front_matter dictionary and optionally includes a weight
    (sort order) as stored in site_hierarhcies.

    Args:
        base_dir (string): The base direcory that together with dir_levels
            represent the directories where the _index files
            are created
        dir_levels (dataframe): The various directory levels that are joined
            to base_dir
        site_hierarchies (WebPageHierarhies): Object storing the web page 
            hierarchies and sort order
                        
        front_matter (dictionary, optional): Optional front matter text to be
            stored in the _index.md files.
            Defaults to None.
    """ 
    if front_matter is None:
        front_matter = {}
    for i in range(1, len(dir_levels.columns)+1):
        # Looping through various levels in directory structure
        subdirs_df = dir_levels.iloc[:, :i].drop_duplicates()
        for _, row in subdirs_df.iterrows():
            file_name = (base_dir + os.path.sep +
                         os.path.sep.join(row) + os.path.sep + '_index.md')
            if not os.path.isfile(file_name):
                # Don't overwrite file if it exists
                # Use a copy on each loop to start fresh and not
                # carry over any details from previos loop
                front_matter_to_write = front_matter.copy()
                if sort_orders_df is not None:
                    sort_order = lookup_list_in_df(
                        sort_orders_df, list(row.values))
                    if sort_order is not None:
                        front_matter_to_write['weight'] = sort_order + 1
                string_to_write = get_front_matter_string(
                    front_matter_to_write)
                with open(file_name, "w", encoding="utf-8") as text_file:
                    text_file.write(string_to_write)

1